In [1]:
%pylab inline
import torch
import numpy
import sklearn.datasets
import torchvision.datasets
import torchvision.transforms
import os

torch.manual_seed(0)

Populating the interactive namespace from numpy and matplotlib


In [2]:
# # Preparing dataset
# valid_train_split = 0.9

# cats_dogs_data = sorted(os.listdir('train'))
# cats_dogs_data = numpy.array(cats_dogs_data)
# numpy.random.shuffle(cats_dogs_data)

# labels = []
# for index in range(cats_dogs_data.size):
#     if cats_dogs_data[index][:3] == 'cat':
#         labels.append(1.0)
#     else:
#         labels.append(0.0)
# labels = numpy.array(labels)

# valid_train_split *= float(labels.size)
# valid_train_split = int(valid_train_split)

# data_train = cats_dogs_data[:valid_train_split]
# data_valid = cats_dogs_data[valid_train_split:]

# labels_train = labels[:valid_train_split]
# labels_valid = labels[valid_train_split:]

In [3]:
# Бинарная классификация
class CatDogConvNet(torch.nn.Module):
    def __init__(self):
        super(CatDogConvNet, self).__init__()
        self.batch_norm1 = torch.nn.BatchNorm2d(3)
        self.conv1 = torch.nn.Conv2d(3, 16, 3, padding=1)
        self.act1  = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool2d(2, 2)
        self.batch_norm2 = torch.nn.BatchNorm2d(16)
        self.conv2 = torch.nn.Conv2d(16, 32, 3, padding=1)
        self.act2  = torch.nn.ReLU()

        self.pool2 = torch.nn.MaxPool2d(2, 2)
        self.batch_norm3 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 64, 3, padding=1)
        self.act3  = torch.nn.ReLU()
        
        self.batch_norm4 = torch.nn.BatchNorm1d(64*64)
        self.fc1   = torch.nn.Linear(64 * 64, 256)
        self.act4  = torch.nn.Tanh()
        self.batch_norm5 = torch.nn.BatchNorm1d(256)
        self.fc2   = torch.nn.Linear(256, 64)
        self.act5  = torch.nn.Tanh()
        self.batch_norm6 = torch.nn.BatchNorm1d(64)
        self.fc3   = torch.nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.batch_norm2(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        x = self.batch_norm3(x)
        x = self.conv3(x)
        x = self.act3(x)
        
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        x = self.batch_norm4(x)
        x = self.fc1(x)
        x = self.act4(x)
        x = self.batch_norm5(x)
        x = self.fc2(x)
        x = self.act5(x)
        x = self.batch_norm6(x)
        x = self.fc3(x)
        
        return x

In [4]:
cat_dog_net = CatDogConvNet()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(cat_dog_net.parameters(), lr=1.0e-3)

In [15]:
train_folder = './train'

transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(224),              # Преобразует картинку к указанному размеру
         torchvision.transforms.ToTensor()                   # Переводит объект "картинка" к torch.Tensor
    ])
image_folder = torchvision.datasets.ImageFolder(
    train_folder, 
    transforms)

data_loader = torch.utils.data.DataLoader(
    image_folder,
    shuffle=True,
    pin_memory=True,
    batch_size=10,
    num_workers=4)

In [16]:
for epoch_index in range(10):
    for inp,targ in data_loader:
        print inp.shape
        print targ.shape

ValueError: Traceback (most recent call last):
  File "/home/wind/miniconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/wind/miniconda2/lib/python2.7/site-packages/torchvision-0.1.8-py2.7.egg/torchvision/datasets/folder.py", line 67, in __getitem__
    img = self.transform(img)
  File "/home/wind/miniconda2/lib/python2.7/site-packages/torchvision-0.1.8-py2.7.egg/torchvision/transforms.py", line 29, in __call__
    img = t(img)
  File "/home/wind/miniconda2/lib/python2.7/site-packages/torchvision-0.1.8-py2.7.egg/torchvision/transforms.py", line 209, in __call__
    x1 = random.randint(0, w - tw)
  File "/home/wind/miniconda2/lib/python2.7/random.py", line 242, in randint
    return self.randrange(a, b+1)
  File "/home/wind/miniconda2/lib/python2.7/random.py", line 218, in randrange
    raise ValueError, "empty range for randrange() (%d,%d, %d)" % (istart, istop, width)


In [10]:
print torchvision

<module 'torchvision' from '/home/wind/miniconda2/lib/python2.7/site-packages/torchvision-0.1.8-py2.7.egg/torchvision/__init__.py'>


In [ ]:
batch_size = 10
for epoch in range(100):
    order = random.permutation(train_features.shape[0])
    for start_index in range(0, train_features.shape[0], batch_size):
        optimizer.zero_grad()
        
        batch_indice = order[start_index:start_index+batch_size]
#         print batch_indice
        batch_features = train_features[batch_indice, :, :, :]
        
        batch_features = torch.FloatTensor(batch_features)
        batch_labels = torch.LongTensor(train_labels[batch_indice])
        
        features_var = torch.autograd.Variable(batch_features)
        labels_var = torch.autograd.Variable(batch_labels)
        
        preds_var = cifar_net.forward(features_var)
        loss = criterion(preds_var, labels_var)
        
        loss.backward()
        
        optimizer.step()
        
    order = random.permutation(test_features.shape[0])
    
    all_preds = []
    all_trues = []
    for start_index in range(0, test_features.shape[0], batch_size):
        batch_indice = order[start_index:start_index+batch_size]
        batch_features = test_features[batch_indice, :, :, :].astype(float)
        batch_features = torch.FloatTensor(batch_features)
        trues = test_labels[batch_indice]
        
        features_var = torch.autograd.Variable(batch_features)
        
        preds = cifar_net.forward(features_var).data.numpy().argmax(axis=1)
        
#         print preds.shape
#         print trues.shape
        
        all_preds.append(preds)
        all_trues.append(trues)
        
    all_preds = numpy.concatenate(all_preds)
    all_trues = numpy.concatenate(all_trues)
    
    print (all_preds == all_trues).sum() / float(all_trues.size)

0.6546
0.6966
0.7109
0.7207
0.7129
0.716


# Задание 1
Добавить (раскомментировать) строки кода, добавляющие BatchNorm в модель. Улучшается ли при этом результат? Почему?

# Задание 2
Увеличить размер пятна (perception area) каждой свертки. Улучшится ли качество?

# Задание 3 
Можно ли увеличить размер пятна каким-либо другим способом (помимо увеличения размера свертки) в промежутках между MaxPooling?